In [2]:
%pip install spacy

!python -m spacy download fr_core_news_sm


   ---------------------------------------- 0.0/12.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/12.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/12.1 MB 2.1 MB/s eta 0:00:06
    --------------------------------------- 0.2/12.1 MB 1.7 MB/s eta 0:00:07
   - -------------------------------------- 0.3/12.1 MB 1.8 MB/s eta 0:00:07
   - -------------------------------------- 0.4/12.1 MB 1.7 MB/s eta 0:00:07
   - -------------------------------------- 0.5/12.1 MB 2.0 MB/s eta 0:00:06
   -- ------------------------------------- 0.7/12.1 MB 2.3 MB/s eta 0:00:05
   -- ------------------------------------- 0.7/12.1 MB 2.1 MB/s eta 0:00:06
   -- ------------------------------------- 0.9/12.1 MB 2.3 MB/s eta 0:00:05
   ---- ----------------------------------- 1.2/12.1 MB 2.6 MB/s eta 0:00:05
   ---- ----------------------------------- 1.4/12.1 MB 2.7 MB/s eta 0:00:04
   ----- ---------------------------------- 1.5/12.1 MB 2.8 MB/s eta 0:00:04
   ----- ---

Les imports

In [3]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
#from sklearn.metrics import confusion_matrix
#from matplotlib import pyplot as plt
#import seaborn as sn
import spacy
from imblearn.over_sampling import SMOTE

creation des dataframe pour entrainement et test

In [4]:
df_test = pd.read_csv("../data/test/test.csv")
df_train = pd.read_csv("../data/train/train.csv")
dff = pd.read_csv("../data/test.csv")

In [5]:
df_train.head()

num_rows = len(df_train)
print("Number of rows:", num_rows)

df_train['type'].value_counts()

Number of rows: 9977


type
Plat principal    4641
Dessert           3009
Entrée            2327
Name: count, dtype: int64

In [6]:
df_test.head()

num_rows = len(df_test)
print("Number of rows:", num_rows)

df_test['type'].value_counts()

Number of rows: 2496


type
Plat principal    1161
Dessert            753
Entrée             582
Name: count, dtype: int64

Gerer l'imbalance de classe Avec du oversampling et du undersampling


Division des datasets en X_train, X_test, Y_train, Y_test. 

In [5]:
X_train = df_train['recette']
X_test = df_test['recette']

Y_train = df_train['type']
Y_test = df_test['type']


test smote

In [10]:
# Step 1: TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(df_train['recette'])

# Step 2: Instantiate SMOTE
smote = SMOTE(random_state=42)

# Step 3: Apply SMOTE to your training data
X_train_resampled, Y_train_resampled = smote.fit_resample(X_train_tfidf, df_train['type'])

# Step 4: Create a pipeline object
clf = Pipeline([
    ('Random Forest', RandomForestClassifier())
])

# Step 5: Fit the pipeline with resampled X_train and Y_train
clf.fit(X_train_resampled, Y_train_resampled)

# Step 6: Transform the test data using the same TF-IDF vectorizer
X_test_tfidf = tfidf_vectorizer.transform(df_test['recette'])

# Step 7: Get the predictions for X_test and store it in Y_pred
Y_pred = clf.predict(X_test_tfidf)

# Step 8: Print the classification report
print(classification_report(df_test['type'], Y_pred))

                precision    recall  f1-score   support

       Dessert       0.97      0.99      0.98       753
        Entrée       0.72      0.52      0.60       582
Plat principal       0.79      0.89      0.84      1161

      accuracy                           0.84      2496
     macro avg       0.83      0.80      0.81      2496
  weighted avg       0.83      0.84      0.83      2496



TFI-DF + KNN

In [29]:
#1. create a pipeline object
clf = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),    
     ('KNN', KNeighborsClassifier())         
])

#2. fit with X_train and y_train
clf.fit(X_train, Y_train)


#3. get the predictions for X_test and store it in y_pred
Y_pred = clf.predict(X_test)


#4. print the classfication report
print(classification_report(Y_test, Y_pred))



                precision    recall  f1-score   support

       Dessert       0.89      0.96      0.92       753
        Entrée       0.67      0.47      0.55       582
Plat principal       0.78      0.86      0.82      1161

      accuracy                           0.80      2496
     macro avg       0.78      0.76      0.76      2496
  weighted avg       0.79      0.80      0.79      2496



TFI-DF + Naive bayes

In [30]:
#1. create a pipeline object
clf = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),    
     ('Multi NB', MultinomialNB())         
])

#2. fit with X_train and y_train
clf.fit(X_train, Y_train)


#3. get the predictions for X_test and store it in y_pred
Y_pred = clf.predict(X_test)


#4. print the classfication report
print(classification_report(Y_test, Y_pred))



                precision    recall  f1-score   support

       Dessert       0.97      0.95      0.96       753
        Entrée       0.94      0.06      0.11       582
Plat principal       0.67      1.00      0.80      1161

      accuracy                           0.76      2496
     macro avg       0.86      0.67      0.63      2496
  weighted avg       0.83      0.76      0.69      2496



TFI-DF + Random Forest

In [31]:
#1. create a pipeline object
clf = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),        #using the ngram_range parameter 
     ('Random Forest', RandomForestClassifier())         
])

#2. fit with X_train and y_train
clf.fit(X_train, Y_train)


#3. get the predictions for X_test and store it in y_pred
Y_pred = clf.predict(X_test)


#4. print the classfication report
print(classification_report(Y_test, Y_pred))



                precision    recall  f1-score   support

       Dessert       0.96      0.99      0.98       753
        Entrée       0.85      0.30      0.44       582
Plat principal       0.74      0.96      0.84      1161

      accuracy                           0.82      2496
     macro avg       0.85      0.75      0.75      2496
  weighted avg       0.83      0.82      0.79      2496



Normalisation des recette

In [7]:
# Load French language model and create nlp object from it
nlp = spacy.load("fr_core_news_sm")

def preprocess(text):
    # Remove stop words, convert to lowercase, and lemmatize the text
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        # Customize filtering conditions for French text
        if token.is_stop or token.is_punct:
            continue
        # Convert token to lowercase and append to filtered_tokens
        filtered_tokens.append(token.lemma_.lower())
    
    return " ".join(filtered_tokens)

In [8]:
df_train['preprocessed_txt'] = df_train['recette'].apply(preprocess)
df_test['preprocessed_txt'] = df_test['recette'].apply(preprocess)



In [9]:
df_train.head()

,doc_id,titre,type,difficulte,cout,ingredients,recette,preprocessed_txt
0,recette_19924.xml,Bûche de Noël pistachée,Dessert,Moyennement difficile,Moyen,- Pour le biscuit : - 4 œufs + 2 jaunes - 110 ...,Préchauffez le four à (210°C). Étalez du papie...,préchauffez four 210 degré c étaler papier sul...
1,recette_25149.xml,Tarte aux pommes au fromage blanc,Dessert,Facile,Moyen,- 1 rouleau de pâte feuilletée ou brisée - 3 p...,"Pelez et coupez les pommes en quartiers, retir...",peler couper pomme quartier retirer pépin fait...
2,recette_40068.xml,Salade de chèvre chaud au miel et pesto,Entrée,Très facile,Bon marché,- 4 chèvres frais - 4 poignées de roquette (ou...,Faire une couronne avec la roquette dans chaqu...,faire couronne roquette assiette déposer tomat...
3,recette_62942.xml,Pommes de terre farcies aux escargots,Entrée,Très facile,Moyen,- 2 très grosses pommes de terre - 4 douzaines...,Faire cuire les pommes de terre à l'eau mais p...,faire cuire pomme terre eau complètement 15 20...
4,recette_12714.xml,Tarte au fromage et à l'abricot,Dessert,Facile,Bon marché,- Pour la pâte : - 200 g de farine - 1 oeuf - ...,Coupez en petit dés le beurre que vous mélange...,couper petit der beurre mélanger sucre oeuf fa...


In [10]:
X_train = df_train['preprocessed_txt']
X_test = df_test['preprocessed_txt']

Y_train = df_train['type']
Y_test = df_test['type']

In [11]:
# Step 1: TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(df_train['preprocessed_txt'])

# Step 2: Instantiate SMOTE
smote = SMOTE(random_state=42)

# Step 3: Apply SMOTE to your training data
X_train_resampled, Y_train_resampled = smote.fit_resample(X_train_tfidf, df_train['type'])

# Step 4: Create a pipeline object
clf = Pipeline([
    ('Random Forest', RandomForestClassifier())
])

# Step 5: Fit the pipeline with resampled X_train and Y_train
clf.fit(X_train_resampled, Y_train_resampled)

# Step 6: Transform the test data using the same TF-IDF vectorizer
X_test_tfidf = tfidf_vectorizer.transform(df_test['preprocessed_txt'])

# Step 7: Get the predictions for X_test and store it in Y_pred
Y_pred = clf.predict(X_test_tfidf)

# Step 8: Print the classification report
print(classification_report(df_test['type'], Y_pred))

                precision    recall  f1-score   support

       Dessert       0.97      0.99      0.98       753
        Entrée       0.70      0.51      0.59       582
Plat principal       0.79      0.89      0.84      1161

      accuracy                           0.83      2496
     macro avg       0.82      0.80      0.80      2496
  weighted avg       0.82      0.83      0.82      2496



TF-IDF + KNN

In [43]:
#1. create a pipeline object
clf = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),    
     ('KNN', KNeighborsClassifier())         
])

#2. fit with X_train and y_train
clf.fit(X_train, Y_train)


#3. get the predictions for X_test and store it in y_pred
Y_pred = clf.predict(X_test)


#4. print the classfication report
print(classification_report(Y_test, Y_pred))

                precision    recall  f1-score   support

       Dessert       0.88      0.95      0.91       753
        Entrée       0.67      0.52      0.59       582
Plat principal       0.79      0.84      0.82      1161

      accuracy                           0.80      2496
     macro avg       0.78      0.77      0.77      2496
  weighted avg       0.79      0.80      0.79      2496



TF-IDF + Naive Bayes

In [42]:
#1. create a pipeline object
clf = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),    
     ('Multi NB', MultinomialNB())         
])

#2. fit with X_train and y_train
clf.fit(X_train, Y_train)


#3. get the predictions for X_test and store it in y_pred
Y_pred = clf.predict(X_test)


#4. print the classfication report
print(classification_report(Y_test, Y_pred))

                precision    recall  f1-score   support

       Dessert       0.97      0.97      0.97       753
        Entrée       0.93      0.16      0.27       582
Plat principal       0.70      0.99      0.82      1161

      accuracy                           0.79      2496
     macro avg       0.87      0.71      0.69      2496
  weighted avg       0.84      0.79      0.74      2496



TF-IDF + Random Forest

In [46]:
#1. create a pipeline object
clf = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),       
     ('Random Forest', RandomForestClassifier())         
])

#2. fit with X_train and y_train
clf.fit(X_train, Y_train)


#3. get the predictions for X_test and store it in y_pred
Y_pred = clf.predict(X_test)


#4. print the classfication report
print(classification_report(Y_test, Y_pred))

                precision    recall  f1-score   support

       Dessert       0.97      0.99      0.98       753
        Entrée       0.83      0.36      0.50       582
Plat principal       0.75      0.96      0.84      1161

      accuracy                           0.83      2496
     macro avg       0.85      0.77      0.77      2496
  weighted avg       0.83      0.83      0.80      2496



In [48]:
wrong_indices = [i for i, (pred, actual) in enumerate(zip(Y_pred, Y_test)) if pred != actual]

results_list = []

# Populate the list with incorrectly classified instances
for index in wrong_indices:
    results_list.append({'Index': index, 
                         'Predicted': Y_pred[index],
                         'Actual': Y_test[index]})

# Convert the list to a DataFrame
results_df = pd.DataFrame(results_list)

# Print the DataFrame
print("Incorrectly classified instances:")
results_df

Incorrectly classified instances:


,Index,Predicted,Actual
0,1,Plat principal,Entrée
1,4,Plat principal,Entrée
2,15,Entrée,Plat principal
3,19,Plat principal,Entrée
4,23,Plat principal,Entrée
...,...,...,...
428,2464,Plat principal,Entrée
429,2476,Entrée,Dessert
430,2478,Entrée,Plat principal
431,2485,Plat principal,Entrée
